In [3]:
from Bio import PDB
import numpy as np
import io
import pandas as pd

In [4]:
seq_R5 = "MSEEAQNDPLLPGYSFNAHLVAGLTPIEANGYLDFFIDRPLGMKGYILNLTIRGQGVVKNQGREFVCRPGDILLFPPGEIHHYGRHPEAREWYHQWVYFRPRAYWHEWLNWPSIFANTGFFRPDEAHQPHFSDLFGQIINAGQGEGRYSELLAINLIEQLLLRRMEAINESSGLATTLERIEKNFVITDPRLPDNPIIFASDSFLQLTEYSREEILGRNCRFLQGPETDRATVRKIRDAIDNQTEVTVQLINYTKSGKKFWNLFHLQPMRDQKGDVQYFIGVQLDGTEHVRDAAEREGVMLIKKTAENIDEAAKGSLHPPMDNRVREACQYISDHLADSNFDIASVAQHVCLSPSRLSHLFRQQLGISVLSWREDQRISQAKLLLSTTRMPIATVGHNVGFDDQLYFSRVFKKCTGASPSEFRAGCEEKVNDVAVKLSG"

In [16]:
array = pd.read_csv(f"/home/student/anna/DMS_analysis/output/Nanopore/pymol/total_enrichment_R5_Bcs_Q20_treshold5e-2.csv", index_col=False, dtype = np.float64)#("output/Mut1_coverage.csv", header=None).values.flatten()

## keep all columns that are enriched at least once 
array = array.sum()
array[array > 1] = 1

array = np.round(array, 5) * 100

In [6]:
def overwrite_b_factors(pdb_str: str, bfactors: np.ndarray) -> str:
    """From the original AF2 github: Overwrites the B-factors in pdb_str with contents of bfactors array."""

    parser = PDB.MMCIFParser(QUIET=True)
    handle = io.StringIO(pdb_str)
    structure = parser.get_structure('', handle)

    curr_resid = ('', '', '')
    idx = -1
    for atom in structure.get_atoms():
        atom_resid = atom.parent.get_id()
        if atom_resid != curr_resid:
            idx += 1
            if idx >= bfactors.shape[0]:
                raise ValueError('Index into bfactors exceeds number of residues. '
                                 'B-factors shape: {shape}, idx: {idx}.')
        curr_resid = atom_resid
        atom.bfactor = bfactors[idx]

    new_pdb = io.StringIO()
    pdb_io = PDB.PDBIO()
    pdb_io.set_structure(structure)
    pdb_io.save(new_pdb)
    return new_pdb.getvalue()

In [21]:
path = "/home/student/anna/DMS_analysis/data/pymol/alphafold_arac_s170_lov2_wt_dimer/fold_arac_s170_lov2_wt_dimer_model_0.cif"#'/Users/annavonbachmann/Downloads/fold_2025_02_05_13_34_my_first_job/fold_2025_02_05_13_34_my_first_job_model_0.cif'

In [22]:
with open(path,'r') as f:
    pdb_str = f.read()
    

In [23]:
# parser = PDB.MMCIFParser(QUIET=True)
# handle = io.StringIO(pdb_str)
# structure = parser.get_structure('', handle)

In [23]:
## extend array at the end by array, if dimer
array = np.append(array, array)

In [24]:
new_pdb_str = overwrite_b_factors(pdb_str,array)

In [25]:
with open("/home/student/anna/DMS_analysis/data/pymol/alphafold_arac_s170_lov2_wt_dimer/colored_after_Nanopore/fold_arac_s170_lov2_wt_dimer_model_0_colored_after_R5_enriched_regions_min5e-1_combined_barcodes.pdb", 'w') as f:
    f.write(new_pdb_str) #'/Users/annavonbachmann/Downloads/fold_2025_02_05_13_34_my_first_job/fold_2025_02_05_13_34_my_first_job_model_0_Mut1_coverage.pdb'

### pymol arguments
alignment 

`align pdbfilename1 filename2`


set coloring binary grey, blue

`spectrum b, grey blue, minimum=0, maximum=100`

`spectrum b, gray70 marine, minimum=0, maximum=100`


movie: 

`mset 1x160` then set and choose the right frame at the bottom